# Scala Version

In [2]:
%scala 
val cable_df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("/FileStore/tables/cable-3.csv")

display(cable_df)

viewing_timestamp,household_id,ad_id
2020-08-12 20:01:20,1,1001
2020-08-17 22:22:15,1,1001
2020-08-21 05:33:33,1,1002
2020-08-22 22:52:28,2,1001
2020-08-30 17:02:06,2,1002


In [3]:
%scala
cable_df.printSchema()

root
-- viewing_timestamp: string (nullable = true)
-- household_id: integer (nullable = true)
-- ad_id: integer (nullable = true)

In [4]:
%scala 
val ott_df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("/FileStore/tables/ott-3.csv")

display(ott_df)

viewing_timestamp,household_id,ad_id
2020-08-22 22:23:12,1,1002
2020-08-22 13:34:32,2,1001
2020-08-28 05:42:18,2,1002
2020-08-29 21:12:16,3,1001


In [5]:
%scala 
val ads_df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("/FileStore/tables/ads-3.csv")

display(ads_df)

ad_id,advertiser
1001,Nike
1002,Nike
1003,Coke


In [6]:
%scala

import org.apache.spark.sql.functions.{lit, from_unixtime, unix_timestamp}

val _all = cable_df.withColumn("service", lit("cable")).union(ott_df.withColumn("service", lit("ott"))).union(cable_df.withColumn("service", lit("both"))).union(ott_df.withColumn("service", lit("both"))).withColumn("viewing_month", from_unixtime(unix_timestamp($"viewing_timestamp", "yyyy-MM-dd HH:mm:ss"), "yyyy-MM")).withColumn("hour", from_unixtime(unix_timestamp($"viewing_timestamp", "yyyy-MM-dd HH:mm:ss"), "HH")).filter(($"hour">=20) && ($"hour"<=22))

val result_df = _all.join(ads_df, _all("ad_id") === ads_df("ad_id")).groupBy("advertiser", "viewing_month", "service").count().as("ads")

// val result_df = _all.join(ads_df, _all("ad_id") === ads_df("ad_id")).repartition(numPartitions = 2, $"advertiser", $"viewing_month", $"service").groupBy("advertiser", "viewing_month", "service").count().as("ads")

result_df.show()

+----------+-------------+-------+-----+
advertiser|viewing_month|service|count|
+----------+-------------+-------+-----+
 Nike| 2020-08| both| 5|
 Nike| 2020-08| cable| 3|
 Nike| 2020-08| ott| 2|
+----------+-------------+-------+-----+

import org.apache.spark.sql.functions.{lit, from_unixtime, unix_timestamp}
_all: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [viewing_timestamp: string, household_id: int ... 4 more fields]
result_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [advertiser: string, viewing_month: string ... 2 more fields]

In [7]:
%scala
// default partition 200
result_df.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[advertiser#165, viewing_month#677, service#637], functions=[finalmerge_count(merge count#744L) AS count(1)#722L])
+- Exchange hashpartitioning(advertiser#165, viewing_month#677, service#637, 200), true, [id=#1513]
 +- *(6) HashAggregate(keys=[advertiser#165, viewing_month#677, service#637], functions=[partial_count(1) AS count#744L])
 +- *(6) Project [service#637, viewing_month#677, advertiser#165]
 +- *(6) BroadcastHashJoin [ad_id#96], [ad_id#164], Inner, BuildRight
 :- Union
 : :- *(1) Project [ad_id#96, cable AS service#637, from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#677]
 : : +- *(1) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#96))
 : : +- FileScan csv [viewing_timestamp#94,ad_id#96] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : :- *(2) Project [ad_id#131, ott AS service#642, from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#783]
 : : +- *(2) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#131))
 : : +- FileScan csv [viewing_timestamp#129,ad_id#131] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), H..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : :- *(3) Project [ad_id#96, both AS service#659, from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#784]
 : : +- *(3) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#96))
 : : +- FileScan csv [viewing_timestamp#94,ad_id#96] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : +- *(4) Project [ad_id#131, both AS service#672, from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#785]
 : +- *(4) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#131))
 : +- FileScan csv [viewing_timestamp#129,ad_id#131] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), H..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>

In [8]:
%scala
// set repartition to 2
result_df.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[advertiser#165, viewing_month#217, service#177], functions=[count(1)])
+- Exchange hashpartitioning(advertiser#165, viewing_month#217, service#177, 2), false, [id=#769]
 +- *(6) Project [service#177, viewing_month#217, advertiser#165]
 +- *(6) BroadcastHashJoin [ad_id#96], [ad_id#164], Inner, BuildRight
 :- Union
 : :- *(1) Project [ad_id#96, cable AS service#177, from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#217]
 : : +- *(1) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#96))
 : : +- FileScan csv [viewing_timestamp#94,ad_id#96] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : :- *(2) Project [ad_id#131, ott AS service#182, from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#326]
 : : +- *(2) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#131))
 : : +- FileScan csv [viewing_timestamp#129,ad_id#131] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), H..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : :- *(3) Project [ad_id#96, both AS service#199, from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#327]
 : : +- *(3) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#96))
 : : +- FileScan csv [viewing_timestamp#94,ad_id#96] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#94, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 : +- *(4) Project [ad_id#131, both AS service#212, from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), yyyy-MM, Some(Etc/UTC)) AS viewing_month#328]
 : +- *(4) Filter (((cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) >= 20) AND (cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), HH, Some(Etc/UTC)) as int) <= 22)) AND isnotnull(ad_id#131))
 : +- FileScan csv [viewing_timestamp#129,ad_id#131] Batched: false, DataFilters: [(cast(from_unixtime(unix_timestamp(viewing_timestamp#129, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC)), H..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:int>
 +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint))), [id=#764]
 +- *(5) Project [ad_id#164, advertiser#165]
 +- *(5) Filter i

In [9]:
%scala
val permanent_table_name = "result_scala"
result_df.write.format("parquet").saveAsTable(permanent_table_name)

at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:685)
	at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:606)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:2)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:88)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:90)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:92)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:94)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:96)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:98)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:100)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:102)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:104)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:106)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:108)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:110)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:112)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:114)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:116)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:118)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:120)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:122)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:124)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:126)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw$$iw.<init>(command-3465052138114507:128)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw$$iw.<init>(command-3465052138114507:130)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw$$iw.<init>(command-3465052138114507:132)
	at linea69ba5e806b3451888794df63395eebd203.$read$$iw.<init>(command-3465052138114507:134)
	at linea69ba5e806b3451888794df63395eebd203.$read.<init>(command-3465052138114507:136)
	at linea69ba5e806b3451888794df63395eebd203.$read$.<init>(command-3465052138114507:140)
	at linea69ba5e806b3451888794df63395eebd203.$read$.<clinit>(command-3465052138114507)
	at linea69ba5e806b3451888794df63395eebd203.$eval$.$print$lzycompute(<notebook>:7)
	at linea69ba5e806b3451888794d

In [10]:
%scala
val read_parquet_df = spark.read.parquet("/user/hive/warehouse/result_scala")
read_parquet_df.show()

+----------+-------------+-------+-----+
advertiser|viewing_month|service|count|
+----------+-------------+-------+-----+
 Nike| 2020-08| cable| 3|
 Nike| 2020-08| both| 5|
 Nike| 2020-08| ott| 2|
+----------+-------------+-------+-----+

read_parquet_df: org.apache.spark.sql.DataFrame = [advertiser: string, viewing_month: string ... 2 more fields]

# PySpark Version

In [12]:
# File location and type
file_location = "/FileStore/tables/cable-3.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
cable_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(cable_df)

viewing_timestamp,household_id,ad_id
2020-08-12 20:01:20,1,1001
2020-08-17 22:22:15,1,1001
2020-08-21 05:33:33,1,1002
2020-08-22 22:52:28,2,1001
2020-08-30 17:02:06,2,1002


In [13]:
cable_df.printSchema()

root
-- viewing_timestamp: string (nullable = true)
-- household_id: string (nullable = true)
-- ad_id: string (nullable = true)

In [14]:
# File location and type
file_location = "/FileStore/tables/ott-3.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
ott_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(ott_df)

viewing_timestamp,household_id,ad_id
2020-08-22 22:23:12,1,1002
2020-08-22 13:34:32,2,1001
2020-08-28 05:42:18,2,1002
2020-08-29 21:12:16,3,1001


In [15]:
# File location and type
file_location = "/FileStore/tables/ads-3.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
ads_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(ads_df)

ad_id,advertiser
1001,Nike
1002,Nike
1003,Coke


In [16]:
from pyspark.sql.functions import year, month, dayofmonth, hour, concat, lit

_all = cable_df.withColumn('service', lit('cable'))\
.union(ott_df.withColumn('service', lit('ott')))\
.union(cable_df.withColumn('service', lit('both')))\
.union(ott_df.withColumn('service', lit('both'))).withColumn('viewing_month', concat(year('viewing_timestamp'), lit("-"), month('viewing_timestamp'))).withColumn('hour', hour('viewing_timestamp'))

result_df = _all.join(ads_df, _all.ad_id == ads_df.ad_id).filter((_all.hour>=20) & (_all.hour<=22)).repartition(2, ["advertiser", "viewing_month", "service"]).groupby(["advertiser", "viewing_month", "service"]).count().alias('ads')
#result_df = _all.join(ads_df, _all.ad_id == ads_df.ad_id).filter((_all.hour>=20) & (_all.hour<=22)).groupby(["advertiser", "viewing_month", "service"]).count().alias('ads')

result_df.show()


+----------+-------------+-------+-----+
advertiser|viewing_month|service|count|
+----------+-------------+-------+-----+
 Nike| 2020-8| ott| 2|
 Nike| 2020-8| both| 5|
 Nike| 2020-8| cable| 3|
+----------+-------------+-------+-----+

In [17]:
# default
result_df.rdd.getNumPartitions()

Out[96]: 200

In [18]:
# repartition
result_df.rdd.getNumPartitions()

Out[98]: 2

In [19]:
result_df.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[advertiser#124, viewing_month#9929, service#9889], functions=[count(1)])
+- Exchange hashpartitioning(advertiser#124, viewing_month#9929, service#9889, 2), false, [id=#19150]
 +- *(6) Project [service#9889, viewing_month#9929, advertiser#124]
 +- *(6) BroadcastHashJoin [ad_id#61], [ad_id#123], Inner, BuildRight
 :- Union
 : :- *(1) Project [ad_id#61, cable AS service#9889, concat(cast(year(cast(viewing_timestamp#59 as date)) as string), -, cast(month(cast(viewing_timestamp#59 as date)) as string)) AS viewing_month#9929]
 : : +- *(1) Filter (((hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) >= 20) AND (hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) <= 22)) AND isnotnull(ad_id#61))
 : : +- FileScan csv [viewing_timestamp#59,ad_id#61] Batched: false, DataFilters: [(hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) >= 20), (hour(cast(viewing_timesta..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:string>
 : :- *(2) Project [ad_id#93, ott AS service#9894, concat(cast(year(cast(viewing_timestamp#91 as date)) as string), -, cast(month(cast(viewing_timestamp#91 as date)) as string)) AS viewing_month#10038]
 : : +- *(2) Filter (((hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) >= 20) AND (hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) <= 22)) AND isnotnull(ad_id#93))
 : : +- FileScan csv [viewing_timestamp#91,ad_id#93] Batched: false, DataFilters: [(hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) >= 20), (hour(cast(viewing_timesta..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:string>
 : :- *(3) Project [ad_id#61, both AS service#9911, concat(cast(year(cast(viewing_timestamp#59 as date)) as string), -, cast(month(cast(viewing_timestamp#59 as date)) as string)) AS viewing_month#10039]
 : : +- *(3) Filter (((hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) >= 20) AND (hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) <= 22)) AND isnotnull(ad_id#61))
 : : +- FileScan csv [viewing_timestamp#59,ad_id#61] Batched: false, DataFilters: [(hour(cast(viewing_timestamp#59 as timestamp), Some(Etc/UTC)) >= 20), (hour(cast(viewing_timesta..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/cable-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:string>
 : +- *(4) Project [ad_id#93, both AS service#9924, concat(cast(year(cast(viewing_timestamp#91 as date)) as string), -, cast(month(cast(viewing_timestamp#91 as date)) as string)) AS viewing_month#10040]
 : +- *(4) Filter (((hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) >= 20) AND (hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) <= 22)) AND isnotnull(ad_id#93))
 : +- FileScan csv [viewing_timestamp#91,ad_id#93] Batched: false, DataFilters: [(hour(cast(viewing_timestamp#91 as timestamp), Some(Etc/UTC)) >= 20), (hour(cast(viewing_timesta..., Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ott-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<viewing_timestamp:string,ad_id:string>
 +- BroadcastExchange HashedRelationBroadcastMode(ArrayBuffer(input[0, string, true])), [id=#19145]
 +- *(5) Project [ad_id#123, advertiser#124]
 +- *(5) Filter isnotnull(ad_id#123)
 +- FileScan csv [ad_id#123,advertiser#124] Batched: false, DataFilters: [isnotnull(ad_id#123)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/ads-3.csv], PartitionFilters: [], PushedFilters: [IsNotNull(ad_id)], ReadSchema: struct<ad_id:string,advertiser:string>

In [20]:
permanent_table_name = "result_df_pyspark"
result_df.write.format("parquet").saveAsTable(permanent_table_name)

In [21]:
read_parquet_df = sqlContext.read.parquet('/user/hive/warehouse/result_df_pyspark')
read_parquet_df.show()

+----------+-------------+-------+-----+
advertiser|viewing_month|service|count|
+----------+-------------+-------+-----+
 Nike| 2020-8| cable| 3|
 Nike| 2020-8| both| 5|
 Nike| 2020-8| ott| 2|
+----------+-------------+-------+-----+

# SQL Version

In [23]:
# Create a view or table

temp_table_name = "ott_csv"

ott_df.createOrReplaceTempView(temp_table_name)

In [24]:
%sql

/* Query the created temp table in a SQL cell */

select * from `ott_csv`

viewing_timestamp,household_id,ad_id
2020-08-22 22:23:12,1,1002
2020-08-22 13:34:32,2,1001
2020-08-28 05:42:18,2,1002
2020-08-29 21:12:16,3,1001


In [25]:
# Create a view or table

temp_table_name = "cable_csv"

cable_df.createOrReplaceTempView(temp_table_name)

In [26]:
%sql

/* Query the created temp table in a SQL cell */

select * from `cable_csv`

viewing_timestamp,household_id,ad_id
2020-08-12 20:01:20,1,1001
2020-08-17 22:22:15,1,1001
2020-08-21 05:33:33,1,1002
2020-08-22 22:52:28,2,1001
2020-08-30 17:02:06,2,1002


In [27]:
# Create a view or table

temp_table_name = "ads_csv"

ads_df.createOrReplaceTempView(temp_table_name)

In [28]:
%sql

/* Query the created temp table in a SQL cell */

select * from `ads_csv`

ad_id,advertiser
1001,Nike
1002,Nike
1003,Coke


In [29]:
%sql
select *, substring(viewing_timestamp, 12 , 2) as hour from `cable_csv` where int(substring(viewing_timestamp, 12 , 2)) >= 20 and int(substring(viewing_timestamp, 12 , 2)) <= 22

viewing_timestamp,household_id,ad_id,hour
2020-08-12 20:01:20,1,1001,20
2020-08-17 22:22:15,1,1001,22
2020-08-22 22:52:28,2,1001,22


In [30]:
sqlContext.sql("set spark.sql.shuffle.partitions=2")

Out[23]: DataFrame[key: string, value: string]

In [31]:
%sql

/* Query the created temp table in a SQL cell */

select advertiser, substring(viewing_timestamp, 0 , 7) as viewing_month, service, count(*) as ads
from 
(
  select *, int(substring(viewing_timestamp, 12 , 2)) as hour from
  (
    select *, 'cable' as service from `cable_csv` union
    select *, 'ott' as service from `ott_csv` union
    select *, 'both' as service from `cable_csv` union
    select *, 'both' as service from `ott_csv`
   ) as _all join `ads_csv` as ad on _all.ad_id = ad.ad_id 
  where int(substring(viewing_timestamp, 12 , 2)) >= 20 and int(substring(viewing_timestamp, 12 , 2)) <= 22 
)
group by advertiser, viewing_month, service


advertiser,viewing_month,service,ads
Nike,2020-08,cable,3
Nike,2020-08,ott,2
Nike,2020-08,both,5


In [32]:
result_df = sqlContext.sql(\
                           "select advertiser, substring(viewing_timestamp, 0 , 7) as viewing_month, service, count(*) as ads\
                            from \
                            (\
                              select *, int(substring(viewing_timestamp, 12 , 2)) as hour from\
                              (\
                                select *, 'cable' as service from `cable_csv` union\
                                select *, 'ott' as service from `ott_csv` union\
                                select *, 'both' as service from `cable_csv` union\
                                select *, 'both' as service from `ott_csv`\
                               ) as _all join `ads_csv` as ad on _all.ad_id = ad.ad_id \
                              where int(substring(viewing_timestamp, 12 , 2)) >= 20 and int(substring(viewing_timestamp, 12 , 2)) <= 22 \
                            )\
                            group by advertiser, viewing_month, service"\
                          )
result_df.show()

+----------+-------------+-------+---+
advertiser|viewing_month|service|ads|
+----------+-------------+-------+---+
 Nike| 2020-08| both| 5|
 Nike| 2020-08| cable| 3|
 Nike| 2020-08| ott| 2|
+----------+-------------+-------+---+

In [33]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "result_sql"
result_df.write.format("parquet").saveAsTable(permanent_table_name)